In [2]:
import os
import numpy as np
import tensorflow as tf


def load_minibatch(index, var):
    shape = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_shape.npy'))
    for i in range(len(shape)):
        if var.shape[i] == None:
            continue
        elif var.shape[i] != shape[i]:
            raise ValueError('Shape of minibatch and variable are incompatible')
    
    indices = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_indices.npy'))
    values = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_values.npy'))
    sparse_tensor = tf.sparse.SparseTensor(indices=indices, values=values, dense_shape=shape)
    var.assign(tf.cast(tf.sparse.to_dense(sparse_tensor), dtype=tf.float32))

In [3]:
var = tf.Variable(tf.zeros([500, 30, 10002]), dtype=tf.float32, shape=[None, 30, 10002])

load_minibatch(0, var)

In [4]:
import os
import json

with open(os.path.join('DATA', 'numberize.json'), 'r') as f:
    numberize = json.load(f)
        
with open(os.path.join('DATA', 'reverse_numberize.json'), 'r') as f:
    reverse_numberize = json.load(f)

In [5]:
def verbalize_sentence(numberized_sentence):
    words = [reverse_numberize[str(word_number)] for word_number in numberized_sentence if word_number != 10001]
    return ' '.join(words)

def verbalize(vectorized_sentences):
    numberized_sentences = tf.argmax(vectorized_sentences, axis=2).numpy()
    return list(map(verbalize_sentence, numberized_sentences))

In [35]:
import numpy as np

class BatchGenerator(object):

    def __init__(self, num_batches, hidden_size):
        self.num_batches = num_batches
        self.current_idx = 0
        self.x = tf.Variable(np.zeros([500, 30, 10002]), dtype=tf.float32, shape=[None, 30, 10002])
        
    def generate(self):
        while True:
            load_minibatch(self.current_idx, self.x)
            self.current_idx = (self.current_idx + 1) % self.num_batches
            X = np.stack([np.zeros([500, 10002])] + [self.x[:, i, :].numpy() for i in range(29)], axis=1)
            Y = [self.x[:, i, :].numpy() for i in range(30)]
            hidden_state_0 = np.zeros((500, hidden_size))
            cell_state_0 = np.zeros((500, hidden_size))
            yield [[X, hidden_state_0, cell_state_0], Y]
            
num_batches = 6807
batch_generator = BatchGenerator(6807, 128).generate()

In [7]:
from keras import Model, Input
from keras.layers import Dense, Activation, Dropout, LSTM, Lambda, Reshape
from keras.optimizers import Adam

sequence_length = 30
vocab_size = 10002
hidden_size = 128

Reshape_cell = Reshape((1, vocab_size)) 
LSTM_cell = LSTM(hidden_size, return_state = True)
Dense_cell = Dense(vocab_size, activation='softmax')

def LSTM_model(sequence_length, hidden_size, vocab_size):
    """
    Arguments:
    sequence_length -- length of input sequences
    hidden_size -- number of hidden units in LSTM cell
    vocab_size -- size of vocabulary in corpus 
    
    Returns:
    a keras model
    """
    
    # Define inputs to LSTM
    X = Input(shape=(sequence_length, vocab_size), name='sequence_input')
    hidden_state_0 = Input(shape=(hidden_size,), name='hidden_state_0')
    cell_state_0 = Input(shape=(hidden_size,), name='cell_state_0')
    
    hidden_state = hidden_state_0
    cell_state = cell_state_0
    outputs = []
    for t in range(sequence_length):
        # Select t-th time slice
        x = Lambda(lambda y: y[:, t, :])(X)
        # Rehapse and apply LSTM_cell to time slice
        x = Reshape_cell(x)
        hidden_state, _, cell_state = LSTM_cell(inputs=x, initial_state=[hidden_state, cell_state])
        # Apply Dense_cell to output of LSTM
        output = Dense_cell(hidden_state)
        # Step 2.E: add the output to "outputs"
        outputs.append(output)
    
    # Step 3: Create model instance
    model = Model(inputs=[X, hidden_state_0, cell_state_0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

model = LSTM_model(sequence_length, hidden_size, vocab_size)
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Using TensorFlow backend.


In [8]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence_input (InputLayer)     (None, 30, 10002)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 10002)        0           sequence_input[0][0]             
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 10002)     0           lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
                                                                 lambda_3[0][0]                   
                                                                 lambda_4[0][0]             

In [ ]:
batches_to_process = 10

model.fit_generator(batch_generator, batches_to_process, 1)

Epoch 1/1
 3/10 [========>.....................] - ETA: 2:25 - loss: 158.8686 - dense_1_loss: 1.0417 - dense_1_accuracy: 0.0000e+00 - dense_1_accuracy_1: 0.0000e+00 - dense_1_accuracy_2: 0.0033 - dense_1_accuracy_3: 0.0100 - dense_1_accuracy_4: 0.0180 - dense_1_accuracy_5: 0.0380 - dense_1_accuracy_6: 0.0520 - dense_1_accuracy_7: 0.0800 - dense_1_accuracy_8: 0.1027 - dense_1_accuracy_9: 0.1340 - dense_1_accuracy_10: 0.1653 - dense_1_accuracy_11: 0.1993 - dense_1_accuracy_12: 0.2360 - dense_1_accuracy_13: 0.2807 - dense_1_accuracy_14: 0.3267 - dense_1_accuracy_15: 0.3793 - dense_1_accuracy_16: 0.4240 - dense_1_accuracy_17: 0.4740 - dense_1_accuracy_18: 0.5233 - dense_1_accuracy_19: 0.5747 - dense_1_accuracy_20: 0.6220 - dense_1_accuracy_21: 0.6627 - dense_1_accuracy_22: 0.7147 - dense_1_accuracy_23: 0.7640 - dense_1_accuracy_24: 0.8127 - dense_1_accuracy_25: 0.8547 - dense_1_accuracy_26: 0.9020 - dense_1_accuracy_27: 0.9360 - dense_1_accuracy_28: 0.9713 - dense_1_accuracy_29: 1.0000